In [1]:
from datasets import load_dataset
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig

/opt/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
snli = load_dataset("snli")
for split in snli:
    snli[split] = snli[split].filter(lambda x: x["label"] >= 0)

In [4]:
#Load the tokenizer and model
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)

# Tokenize the dataset
encoded_snli = snli.map(tokenize_function, batched=True)
    
# Specify sample size
sample_size = 2000

# Ensure reproducibility
random.seed(68)

# Randomly sample 10 subsets
subsets = []
for idx in range(10):
  subset = encoded_snli['train'].shuffle(seed=68+9-idx).select(range(sample_size))
  subsets.append(subset)

Map: 100%|██████████| 549367/549367 [01:50<00:00, 4953.04 examples/s]


In [6]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}

In [9]:
def train_model(training_data):
  for idx in range(len(training_data)):
    subset = training_data[idx]

    training_args = TrainingArguments(
      output_dir= "".join(["./results", str(idx)]),
      evaluation_strategy="epoch",
      learning_rate=2e-05,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      num_train_epochs=1,
      weight_decay=0.01,
      save_strategy="epoch",
      load_best_model_at_end=True,
      metric_for_best_model="accuracy",)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=subset,
        eval_dataset=encoded_snli["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics)

    trainer.train()


In [10]:
train_model(subsets)

/opt/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/ty/mztdvdnn2l72hll5mys82f3c0000gn/T/ipykernel_10260/1042977821.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
                                                 
100%|██████████| 125/125 [08:49<00:00,  2.14s/it]

{'eval_loss': 1.0591257810592651, 'eval_accuracy': 0.4470636049583418, 'eval_f1': 0.4174311815515342, 'eval_precision': 0.5277979180476682, 'eval_recall': 0.4470636049583418, 'eval_runtime': 262.128, 'eval_samples_per_second': 37.547, 'eval_steps_per_second': 2.35, 'epoch': 1.0}


100%|██████████| 125/125 [08:54<00:00,  4.28s/it]
/opt/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/ty/mztdvdnn2l72hll5mys82f3c0000gn/T/ipykernel_10260/1042977821.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'train_runtime': 534.4926, 'train_samples_per_second': 3.742, 'train_steps_per_second': 0.234, 'train_loss': 1.0857064208984375, 'epoch': 1.0}


                                                 
100%|██████████| 125/125 [09:47<00:00,  2.01s/it]

...

{'train_runtime': 496.8469, 'train_samples_per_second': 4.025, 'train_steps_per_second': 0.252, 'train_loss': 0.6224588012695312, 'epoch': 1.0}


100%|██████████| 125/125 [07:45<00:00,  1.60s/it]

{'eval_loss': 0.5648350715637207, 'eval_accuracy': 0.7763665921560658, 'eval_f1': 0.7748484593190571, 'eval_precision': 0.7750725566945951, 'eval_recall': 0.7763665921560658, 'eval_runtime': 251.8398, 'eval_samples_per_second': 39.08, 'eval_steps_per_second': 2.446, 'epoch': 1.0}


100%|██████████| 125/125 [07:52<00:00,  3.78s/it]

{'train_runtime': 472.0332, 'train_samples_per_second': 4.237, 'train_steps_per_second': 0.265, 'train_loss': 0.6273997802734375, 'epoch': 1.0}
